## やりたいこと
`configpaser.Configpaser`の実装テスト
- iniファイルの生成
- iniファイルからの変数の読み出し

In [1]:
# cording = UTF-8
import os
import configparser

####################################初期化####################################
aug_amount = 512    #ファイルごとのAugmentationの回数 2の階乗値
aug_length = 5      #Augmentation後のデータ長(秒) 音声データより長いこと
wav_sr = 22050      #音声ファイルのサンプリングレート 44100か22050

lr = 2e-02          #AE初期学習率
alpha = 0           #AEL2正則化の係数
dr_rate = 0.2         #AEドロップアウト率
batch_size = 256    #オフライン学習時のバッチサイズ
epochs = 100        #学習時のエポック数(グリッドサーチ時は無効)
encode_dim = 2048   #AE圧縮次元数
hidden_act = 'elu'  #AE隠れ層の活性化関数
out_act = 'linear'  #AE出力層の活性化関数

exp_ver = 0.999     #PCA圧縮率(保持する分散の割合)

monitor = "val_loss"    #学習率減衰/早期打ち切りの指標

#ディレクトリの初期化
base_dir = "../"
data_dir =os.path.join(base_dir,"data")
ok_dir = os.path.join(base_dir,"data/OK")
ng_dir = os.path.join(base_dir,"data/NG")
env_dir = os.path.join(base_dir,"data/Environment")
env_file = "masuho_env.wav"
log_dir = "./logs"       #Tensorboardログの保存パス
modeldir = "./models"   #学習済みモデルの保存パス

#学習用データファイル
dataset_file = "dataset2020.npz"

###########################################################################


カレントディレクトリを確認しておく

In [2]:
os.getcwd()

'c:\\Users\\amane\\OneDrive\\python\\proto2020'

iniファイルを生成する  
`configpaser.Configpaser()`をインスタンス化したのち、セクション名で辞書を作成する

In [3]:
cfg = configparser.ConfigParser()

In [5]:
cfg["General"] = {
    "base_dir" : base_dir,
    "data_dir" : data_dir,
    "dataset_file" : dataset_file,
    "ok_dir" : ok_dir,
    "ng_dir" : ng_dir,
    "env_dir" : env_dir,
    "env_file" : env_file,
    "log_dir" : log_dir,
    "model_dir" :modeldir 
}

cfg["Augmentation"] = {
    "aug_amount" : aug_amount,
    "aug_length" : aug_length,
    "wav_sr" : wav_sr,
}

cfg["model"] = {
    "batch_size" : batch_size
}

cfg["AE"] = {
    "lr" : lr,
    "alpha" : alpha,
    "dr_rate" : dr_rate,
    "epochs" : epochs,
    "encode_dim" : encode_dim,
    "hidden_act" : hidden_act,
    "out_act" : out_act,
    "monitor" : monitor
}

cfg["Psd_AE"] = {
    "exp_ver" : exp_ver
}

In [6]:
cfg.items()

ItemsView(<configparser.ConfigParser object at 0x0000023E6E726E88>)

In [8]:
cfg.items("General")

[('base_dir', '../'),
 ('data_dir', '../data'),
 ('dataset_file', 'dataset2020.npz'),
 ('ok_dir', '../data/OK'),
 ('ng_dir', '../data/NG'),
 ('env_dir', '../data/Environment'),
 ('env_file', 'masuho_env.wav'),
 ('log_dir', './logs'),
 ('model_dir', './models')]

iniファイルの保存。`With`句で`Open`を呼び出して保存する

In [9]:
with open('AE.ini', 'w') as configfile:
    cfg.write(configfile)

In [10]:
#いったん全部消す
del alpha,aug_amount,aug_length,base_dir,batch_size,cfg,configfile,data_dir,dataset_file,dr_rate,encode_dim,env_dir,env_file,epochs,exp_ver,hidden_act,lr,modeldir,monitor,ng_dir,ok_dir,out_act,wav_sr

In [11]:
del log_dir

In [13]:
#iniファイルを読み込む
cfg = configparser.ConfigParser()
cfg.read("./AE.ini")

['./AE.ini']

In [14]:
cfg.items("General")

[('base_dir', '../'),
 ('data_dir', '../data'),
 ('dataset_file', 'dataset2020.npz'),
 ('ok_dir', '../data/OK'),
 ('ng_dir', '../data/NG'),
 ('env_dir', '../data/Environment'),
 ('env_file', 'masuho_env.wav'),
 ('log_dir', './logs'),
 ('model_dir', './models')]

読みだせている

In [16]:
#ほしい変数だけ読みだしてみる
base_dir = cfg.get("General","base_dir")

In [17]:
base_dir

'../'

## わかったこと
- `configpaser.Configpaser.get()`を使えば任意の位置で必要な変数を読み出せる
- `getint()`や`getflort()`で任意のデータ型で読み出せる(デフォルトはstr)
- `configpaser.Configpaser.set()`を使えば新しいキーの定義や既存キーの書き換えができる
- `config@aser.COnfigpaser.read()`でiniファイルはすべて読まれているので、`set()`の後は単純に上書きすればいい

## 次にやること
AE.pyへの実装。ど頭でiniファイルを読み込んで、必要に応じて変数を読みだす。
iniへの書き込みをクラス化しておけば、任意のタイミングで呼び出し可能(なはず)

## 参考
[configpaserの公式ドキュメント](url:https://docs.python.org/ja/3/library/configparser.html)